In [1]:
import pandas as pd
import numpy as np

PATH = '/home/kai/data/kaggle/homecredit/'
# train = pd.read_csv(PATH + 'application_train.csv')
# print('train')
# test = pd.read_csv(PATH + 'application_test.csv')
# print('test')
# bureau = pd.read_csv(PATH + 'bureau.csv')
# print('bureau')
bureau_balance = pd.read_csv(PATH + 'bureau_balance.csv')
print('bureau_balance')
# installments_payments = pd.read_csv(PATH + 'installments_payments.csv')
# print('installments_payments done')
# credit_card_balance = pd.read_csv(PATH + 'credit_card_balance.csv')
# print('credit_card_balance')
# POS_CASH_balance = pd.read_csv(PATH + 'POS_CASH_balance.csv')
# print('POS_CASH_balance')
# credit_card_balance = pd.read_csv(PATH + 'credit_card_balance.csv')
# print('credit_card_balance')
# previous_application = pd.read_csv(PATH + 'previous_application.csv')
# print('previous_application')

bureau_balance


In [3]:
most_recent_index = bureau_balance.groupby('SK_ID_BUREAU')['MONTHS_BALANCE'].idxmax()
most_recent_index

KeyboardInterrupt: 

In [ ]:
bureau_balance = bureau_balance.loc[most_recent_index, :]

In [ ]:
type(most_recent_index)

In [2]:
df = pd.concat([train,test], axis = 0)
df = df[['SK_ID_CURR']]
install = installments_payments.copy()

In [3]:
import pandas as pd
import os
import gc
import numpy as np

def ratio_name(numerator, denominator): return numerator + '_divide_' + denominator

def ratio(df, numerator, denominator):
    return df[numerator] / df[denominator]

def substraction_name(col1, col2): return col1 + '_minus_' + col2

def substraction(df, col1, col2):
    return df[col1] - df[col2]



# Installments: substraction and ratio, late days and score

In [4]:
install.columns.values

array(['SK_ID_PREV', 'SK_ID_CURR', 'NUM_INSTALMENT_VERSION',
       'NUM_INSTALMENT_NUMBER', 'DAYS_INSTALMENT', 'DAYS_ENTRY_PAYMENT',
       'AMT_INSTALMENT', 'AMT_PAYMENT'], dtype=object)

In [5]:
install[substraction_name('DAYS_INSTALMENT', 'DAYS_ENTRY_PAYMENT')] =\
substraction(install, 'DAYS_INSTALMENT' ,'DAYS_ENTRY_PAYMENT')

install[substraction_name('AMT_PAYMENT', 'AMT_INSTALMENT')] = \
substraction(install, 'AMT_PAYMENT' ,'AMT_INSTALMENT')

install[ratio_name('AMT_PAYMENT', 'AMT_INSTALMENT')] = \
ratio(install, 'AMT_PAYMENT' ,'AMT_INSTALMENT')

install['late_days'] = 0
x = install['DAYS_INSTALMENT_minus_DAYS_ENTRY_PAYMENT'] < 0
install['late_days'][x] = install['DAYS_INSTALMENT_minus_DAYS_ENTRY_PAYMENT'][x]

/home/kai/anaconda3/envs/tf_newer/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [7]:
install['score'] = (install['AMT_PAYMENT_divide_AMT_INSTALMENT']*(-20) + 21)\
* np.exp((install['late_days'])**3/100)* (1/(1 + abs(install['DAYS_ENTRY_PAYMENT']/30) ))

# Numerical Columns:

In [11]:
def numerical_agg(df, gp_col, agg_col, df_name, extrafunc_list = None):
    
    agg_list = ['max', 'min', 'std','mean']
    '''
    Possible extrafunc_list: ['sum','median','two_minus_one_third','positive_count', 'negative_count','standard_error', 'trimmed_mean_10_pct', 'trimmed_mean_25_pct','normed_std', 'max_minus_min','one_third','two_third']
    '''
    if extrafunc_list:
        if 'one_third' in extrafunc_list:
            def one_third(series):
                return series.quantile(1/3)
            
        if 'two_third' in extrafunc_list:
            def two_third(series):
                return series.quantile(2/3)
            
        if 'max_minus_min' in extrafunc_list:
            def max_minus_min(series):
                return (series.max() - series.min())
        
        if 'two_minus_one_third' in extrafunc_list:
#             print('two_minus_one')
#             print('get two minus one')
            def two_minus_one_third(series):
                return(series.quantile(0.66666) - series.quantile(0.33333))
       
        if 'positive_count' in extrafunc_list:
#             print('get positive count')
            def positive_count(series):
                return pd.Series(series > 0).sum()

        if 'negative_count' in extrafunc_list:
#             print('get negative count')
            def negative_count(series):
                return pd.Series(series < 0).sum()

        if 'standard_error' in extrafunc_list:
#             print('get std-error')
            def standard_error(series):
                return series.std()/np.sqrt(len(series))

        if 'normed_std' in extrafunc_list:
#             print('get normed_std')
            def normed_std(series):
                return series.std()/series.mean()

        if 'trimmed_mean_10_pct' in extrafunc_list:
#             print('get_trimmed 10 pct')
            def trimmed_mean_10_pct(series):
                return stats.trim_mean(series.dropna(), 0.1)

        if 'trimmed_mean_25_pct' in extrafunc_list:
#             print('get_trimmed 25 pct')
            def trimmed_mean_25_pct(series):
                return stats.trim_mean(series.dropna(), 0.25)
        
        list_tocall = []
        for i in extrafunc_list:
            if i not in set(['sum', 'median']):
                list_tocall.append(eval(i))
            
    if extrafunc_list != None:   
        agg_list.extend(list_tocall)
    
    _df = df.groupby(gp_col).agg({agg_col:agg_list})
    columns = []
    for pre in _df.columns.levels[0]:
        for middle in _df.columns.levels[1]:
            columns.append('%s_%s_%s' %(df_name,pre,middle))
    _df.columns = columns
    
    return _df.reset_index()

In [12]:
install.columns.values

array(['SK_ID_PREV', 'SK_ID_CURR', 'NUM_INSTALMENT_VERSION',
       'NUM_INSTALMENT_NUMBER', 'DAYS_INSTALMENT', 'DAYS_ENTRY_PAYMENT',
       'AMT_INSTALMENT', 'AMT_PAYMENT',
       'DAYS_INSTALMENT_minus_DAYS_ENTRY_PAYMENT',
       'AMT_PAYMENT_minus_AMT_INSTALMENT',
       'AMT_PAYMENT_divide_AMT_INSTALMENT', 'late_days', 'score'],
      dtype=object)

In [13]:
gp_col = 'SK_ID_CURR'
agg_col = ['DAYS_INSTALMENT_minus_DAYS_ENTRY_PAYMENT',
          'AMT_PAYMENT_minus_AMT_INSTALMENT',
           'AMT_PAYMENT_divide_AMT_INSTALMENT',
          'AMT_INSTALMENT',
          'DAYS_ENTRY_PAYMENT',
          'AMT_PAYMENT',
          'late_days','DAYS_INSTALMENT','score']

'''
    Possible extrafunc_list: ['sum','median','two_minus_one_third','positive_count', 'negative_count','standard_error', 'trimmed_mean_10_pct', 'trimmed_mean_25_pct','normed_std', 'max_minus_min','one_third','two_third']
    '''
extrafunc_list = [['negative_count'],['negative_count'],None,None,None,None,None,None,None]
for i in range(len(extrafunc_list)):
    print(agg_col[i])
    df = df.merge(numerical_agg(install, gp_col, agg_col[i], 'installments',extrafunc_list = extrafunc_list[i]),\
                       on = 'SK_ID_CURR', how='left')

DAYS_INSTALMENT_minus_DAYS_ENTRY_PAYMENT
AMT_PAYMENT_minus_AMT_INSTALMENT
AMT_PAYMENT_divide_AMT_INSTALMENT
AMT_INSTALMENT
DAYS_ENTRY_PAYMENT
AMT_PAYMENT
late_days
DAYS_INSTALMENT
score


In [14]:
install.columns.values

array(['SK_ID_PREV', 'SK_ID_CURR', 'NUM_INSTALMENT_VERSION',
       'NUM_INSTALMENT_NUMBER', 'DAYS_INSTALMENT', 'DAYS_ENTRY_PAYMENT',
       'AMT_INSTALMENT', 'AMT_PAYMENT',
       'DAYS_INSTALMENT_minus_DAYS_ENTRY_PAYMENT',
       'AMT_PAYMENT_minus_AMT_INSTALMENT',
       'AMT_PAYMENT_divide_AMT_INSTALMENT', 'late_days', 'score'],
      dtype=object)

In [16]:
print(df.columns.values, len(df.columns.values))

['SK_ID_CURR' 'installments_DAYS_INSTALMENT_minus_DAYS_ENTRY_PAYMENT_max'
 'installments_DAYS_INSTALMENT_minus_DAYS_ENTRY_PAYMENT_min'
 'installments_DAYS_INSTALMENT_minus_DAYS_ENTRY_PAYMENT_std'
 'installments_DAYS_INSTALMENT_minus_DAYS_ENTRY_PAYMENT_mean'
 'installments_DAYS_INSTALMENT_minus_DAYS_ENTRY_PAYMENT_negative_count'
 'installments_AMT_PAYMENT_minus_AMT_INSTALMENT_max'
 'installments_AMT_PAYMENT_minus_AMT_INSTALMENT_min'
 'installments_AMT_PAYMENT_minus_AMT_INSTALMENT_std'
 'installments_AMT_PAYMENT_minus_AMT_INSTALMENT_mean'
 'installments_AMT_PAYMENT_minus_AMT_INSTALMENT_negative_count'
 'installments_AMT_PAYMENT_divide_AMT_INSTALMENT_max'
 'installments_AMT_PAYMENT_divide_AMT_INSTALMENT_min'
 'installments_AMT_PAYMENT_divide_AMT_INSTALMENT_std'
 'installments_AMT_PAYMENT_divide_AMT_INSTALMENT_mean'
 'installments_AMT_INSTALMENT_max' 'installments_AMT_INSTALMENT_min'
 'installments_AMT_INSTALMENT_std' 'installments_AMT_INSTALMENT_mean'
 'installments_DAYS_ENTRY_PAYMENT_max